In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
cd recommenders/

/content/recommenders


In [ ]:
!pip install .

Processing /content/recommenders
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 310 kB 5.1 MB/s 
     |████████████████████████████████| 1.5 MB 53.6 MB/s 
     |████████████████████████████████| 636 kB 56.7 MB/s 
     |████████████████████████████████| 11.8 MB 27.3 MB/s 
     |████████████████████████████████| 12.4 MB 15.4 MB/s 
     |████████████████████████████████| 61 kB 8.6 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 22

In [ ]:
!pip install papermill

     |████████████████████████████████| 155 kB 6.0 MB/s 
     |████████████████████████████████| 125 kB 22.7 MB/s 
     |████████████████████████████████| 843 kB 30.5 MB/s 
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


In [ ]:
!pip install scrapbook

In [ ]:
import sys
import os
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Pandas version: 1.1.5
Tensorflow version: 1.15.2


In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 1024

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "../../recommenders/models/deeprec/config/lightgcn.yaml"
user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

In [ ]:
df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE, title_col="movie_title")

df.head()

100%|██████████| 4.81k/4.81k [00:00<00:00, 16.4kKB/s]


,userID,itemID,rating,timestamp,movie_title
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


In [ ]:
train, test = python_stratified_split(df, ratio=0.75)

In [ ]:
data = ImplicitCF(train=train, test=test, seed=SEED)

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.4s: train loss = 0.46603 = (mf)0.46579 + (embed)0.00025
Epoch 2 (train)0.9s: train loss = 0.27655 = (mf)0.27590 + (embed)0.00065
Epoch 3 (train)0.9s: train loss = 0.24837 = (mf)0.24756 + (embed)0.00081
Epoch 4 (train)0.9s: train loss = 0.23141 = (mf)0.23043 + (embed)0.00098
Epoch 5 (train)0.9s + (eval)0.9s: train loss = 0.22196 = (mf)0.22085 + (embed)0.00111, recall = 0.13091, ndcg = 0.29662, precision = 0.26575, map = 0.06727
Epoch 6 (train)0.9s: train loss = 0.21733 = (mf)0.21612 + (embed)0.00121
Epoch 7 (train)0.9s: train loss = 0.21148 = (mf)0.21018 + (embed)0.00130
Epoch 8 (train)0.9s: train loss = 0.20132 = (mf)0.19991 + (embed)0.00141
Epoch 9 (train)0.8s: train loss = 0.19043 = (mf)0.18887 + (embed)0.00156
Epoch 10 (train)0.9s + (eval)0.2s: train loss = 0.18149 = (mf)0.17975 + (embed)0.00173, recall = 0.14678, ndcg = 0.33228, precision = 0.30255, map = 0.07813
Epoch 11 (train)0.8s: train loss = 0.17338 = (mf)0.17149 + (embed)0.00189
Epoch 12 (train)0.8s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head()

,userID,itemID,prediction
0,1,98,5.605149
1,1,423,5.212590
2,1,69,5.202885
3,1,919,5.160753
4,1,408,5.160677


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.117308
NDCG:	0.421679
Precision@K:	0.374973
Recall@K:	0.193888


In [ ]:
# Record results with papermill for tests
sb.glue("map", eval_map)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)

In [ ]:
model.infer_embedding(user_file, item_file)

In [ ]:
topk_scores

,userID,itemID,prediction
0,1,98,5.605149
1,1,423,5.212590
2,1,69,5.202885
3,1,919,5.160753
4,1,408,5.160677
...,...,...,...
9425,943,550,6.984333
9426,943,405,6.982658
9427,943,88,6.895860
9428,943,144,6.858283


In [ ]:
id_to_title = {}

for i in range(len(df)):
    curr_id = df['itemID'].iloc[i]

    if not curr_id in id_to_title:
        id_to_title[curr_id] = df['movie_title'].iloc[i]

In [ ]:
topk_scores['movie_title'] = ''

In [ ]:
for i in range(len(topk_scores)):
    curr_id = topk_scores['itemID'].iloc[i]
    if curr_id in id_to_title:
        topk_scores['movie_title'].iloc[i] = id_to_title[curr_id]
    else:
        print("No movie title exists for itemID:", curr_id)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
topk_scores.head(50)

,userID,itemID,prediction,movie_title
0,1,98,5.605149,"Silence of the Lambs, The (1991)"
1,1,423,5.212590,E.T. the Extra-Terrestrial (1982)
2,1,69,5.202885,Forrest Gump (1994)
3,1,919,5.160753,"City of Lost Children, The (1995)"
4,1,408,5.160677,"Close Shave, A (1995)"
5,1,403,5.146070,Batman (1989)
6,1,405,5.107685,Mission: Impossible (1996)
7,1,56,5.097847,Pulp Fiction (1994)
8,1,28,5.071700,Apollo 13 (1995)
9,1,91,5.071423,"Nightmare Before Christmas, The (1993)"


In [ ]:
topk_scores.to_csv("top10_lightGCN.csv")

# **Experiments**

## Top k = 20

In [ ]:
# top k items to recommend
TOP_K = 20

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.3s: train loss = 0.46632 = (mf)0.46607 + (embed)0.00024
Epoch 2 (train)0.9s: train loss = 0.27984 = (mf)0.27920 + (embed)0.00064
Epoch 3 (train)0.9s: train loss = 0.24628 = (mf)0.24547 + (embed)0.00081
Epoch 4 (train)0.9s: train loss = 0.23294 = (mf)0.23197 + (embed)0.00097
Epoch 5 (train)0.9s + (eval)0.3s: train loss = 0.22447 = (mf)0.22336 + (embed)0.00110, recall = 0.22034, ndcg = 0.29744, precision = 0.22731, map = 0.09263
Epoch 6 (train)0.9s: train loss = 0.21859 = (mf)0.21738 + (embed)0.00121
Epoch 7 (train)0.9s: train loss = 0.20841 = (mf)0.20709 + (embed)0.00132
Epoch 8 (train)0.9s: train loss = 0.19767 = (mf)0.19622 + (embed)0.00145
Epoch 9 (train)0.9s: train loss = 0.18702 = (mf)0.18542 + (embed)0.00160
Epoch 10 (train)0.9s + (eval)0.2s: train loss = 0.17480 = (mf)0.17304 + (embed)0.00176, recall = 0.25625, ndcg = 0.33916, precision = 0.26241, map = 0.11180
Epoch 11 (train)0.8s: train loss = 0.17110 = (mf)0.16918 + (embed)0.00192
Epoch 12 (train)0.8s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(20)

,userID,itemID,prediction
0,1,432,5.463423
1,1,588,5.325284
2,1,91,5.176545
3,1,423,5.050925
4,1,98,4.976696
5,1,408,4.857318
6,1,455,4.788646
7,1,144,4.734303
8,1,204,4.620700
9,1,69,4.579631


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.158688
NDCG:	0.416448
Precision@K:	0.312195
Recall@K:	0.312891


## Top k = 5

In [ ]:
# top k items to recommend
TOP_K = 5

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.2s: train loss = 0.46508 = (mf)0.46484 + (embed)0.00024
Epoch 2 (train)0.8s: train loss = 0.28421 = (mf)0.28357 + (embed)0.00063
Epoch 3 (train)0.8s: train loss = 0.24773 = (mf)0.24692 + (embed)0.00080
Epoch 4 (train)0.8s: train loss = 0.22829 = (mf)0.22730 + (embed)0.00099
Epoch 5 (train)0.8s + (eval)0.2s: train loss = 0.21923 = (mf)0.21810 + (embed)0.00113, recall = 0.07607, ndcg = 0.32653, precision = 0.31135, map = 0.04954
Epoch 6 (train)0.8s: train loss = 0.21374 = (mf)0.21251 + (embed)0.00123
Epoch 7 (train)0.8s: train loss = 0.20191 = (mf)0.20056 + (embed)0.00135
Epoch 8 (train)0.8s: train loss = 0.19828 = (mf)0.19681 + (embed)0.00148
Epoch 9 (train)0.8s: train loss = 0.18507 = (mf)0.18345 + (embed)0.00161
Epoch 10 (train)0.8s + (eval)0.2s: train loss = 0.17931 = (mf)0.17755 + (embed)0.00177, recall = 0.07944, ndcg = 0.34661, precision = 0.33128, map = 0.05391
Epoch 11 (train)0.8s: train loss = 0.17080 = (mf)0.16888 + (embed)0.00192
Epoch 12 (train)0.9s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(5)

,userID,itemID,prediction
0,1,91,6.189005
1,1,423,5.415013
2,1,250,5.392583
3,1,144,5.386691
4,1,408,5.306597


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.083254
NDCG:	0.449097
Precision@K:	0.431389
Recall@K:	0.115261


## Batch Size = 2048

In [ ]:
TOP_K = 10
# Model parameters
BATCH_SIZE = 2048

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.1s: train loss = 0.59008 = (mf)0.59000 + (embed)0.00008
Epoch 2 (train)0.6s: train loss = 0.31805 = (mf)0.31759 + (embed)0.00046
Epoch 3 (train)0.7s: train loss = 0.28011 = (mf)0.27946 + (embed)0.00065
Epoch 4 (train)0.6s: train loss = 0.25728 = (mf)0.25655 + (embed)0.00073
Epoch 5 (train)0.6s + (eval)0.3s: train loss = 0.24252 = (mf)0.24168 + (embed)0.00084, recall = 0.12512, ndcg = 0.29105, precision = 0.25801, map = 0.06617
Epoch 6 (train)0.7s: train loss = 0.22924 = (mf)0.22829 + (embed)0.00095
Epoch 7 (train)0.7s: train loss = 0.22245 = (mf)0.22140 + (embed)0.00105
Epoch 8 (train)0.6s: train loss = 0.21716 = (mf)0.21602 + (embed)0.00114
Epoch 9 (train)0.6s: train loss = 0.21744 = (mf)0.21624 + (embed)0.00120
Epoch 10 (train)0.7s + (eval)0.2s: train loss = 0.21008 = (mf)0.20881 + (embed)0.00127, recall = 0.13368, ndcg = 0.30810, precision = 0.27667, map = 0.07047
Epoch 11 (train)0.7s: train loss = 0.20741 = (mf)0.20607 + (embed)0.00134
Epoch 12 (train)0.6s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,56,4.993891
1,1,475,4.913111
2,1,98,4.649499
3,1,28,4.470575
4,1,655,4.416765
5,1,273,4.411400
6,1,69,4.375174
7,1,238,4.374312
8,1,405,4.284281
9,1,204,4.272716


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.112982
NDCG:	0.408899
Precision@K:	0.362990
Recall@K:	0.187839


## Batch size = 512

In [ ]:
# Model parameters
BATCH_SIZE = 512

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.7s: train loss = 0.38684 = (mf)0.38644 + (embed)0.00040
Epoch 2 (train)1.2s: train loss = 0.25180 = (mf)0.25101 + (embed)0.00079
Epoch 3 (train)1.3s: train loss = 0.23039 = (mf)0.22936 + (embed)0.00103
Epoch 4 (train)1.3s: train loss = 0.22038 = (mf)0.21917 + (embed)0.00122
Epoch 5 (train)1.3s + (eval)0.3s: train loss = 0.20934 = (mf)0.20797 + (embed)0.00137, recall = 0.13649, ndcg = 0.31833, precision = 0.28558, map = 0.07445
Epoch 6 (train)1.2s: train loss = 0.19423 = (mf)0.19266 + (embed)0.00156
Epoch 7 (train)1.2s: train loss = 0.17984 = (mf)0.17805 + (embed)0.00179
Epoch 8 (train)1.2s: train loss = 0.17448 = (mf)0.17247 + (embed)0.00201
Epoch 9 (train)1.2s: train loss = 0.16200 = (mf)0.15977 + (embed)0.00223
Epoch 10 (train)1.2s + (eval)0.2s: train loss = 0.16087 = (mf)0.15847 + (embed)0.00240, recall = 0.15609, ndcg = 0.34594, precision = 0.31569, map = 0.08486
Epoch 11 (train)1.2s: train loss = 0.15470 = (mf)0.15213 + (embed)0.00257
Epoch 12 (train)1.2s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,403,5.572781
1,1,423,5.415783
2,1,919,5.021138
3,1,582,5.000968
4,1,91,4.929987
5,1,432,4.928307
6,1,652,4.843545
7,1,144,4.836912
8,1,98,4.834475
9,1,455,4.791064


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.117949
NDCG:	0.415721
Precision@K:	0.367762
Recall@K:	0.192301


## Number of layers = 4

In [ ]:
BATCH_SIZE = 1024

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=4,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.5s: train loss = 0.46526 = (mf)0.46500 + (embed)0.00026
Epoch 2 (train)0.9s: train loss = 0.30654 = (mf)0.30592 + (embed)0.00062
Epoch 3 (train)0.9s: train loss = 0.27690 = (mf)0.27614 + (embed)0.00076
Epoch 4 (train)0.9s: train loss = 0.25158 = (mf)0.25063 + (embed)0.00095
Epoch 5 (train)0.9s + (eval)0.3s: train loss = 0.23160 = (mf)0.23044 + (embed)0.00117, recall = 0.12884, ndcg = 0.29607, precision = 0.26299, map = 0.06781
Epoch 6 (train)0.9s: train loss = 0.22714 = (mf)0.22580 + (embed)0.00134
Epoch 7 (train)0.9s: train loss = 0.22324 = (mf)0.22176 + (embed)0.00148
Epoch 8 (train)0.9s: train loss = 0.21391 = (mf)0.21230 + (embed)0.00161
Epoch 9 (train)0.9s: train loss = 0.21036 = (mf)0.20863 + (embed)0.00173
Epoch 10 (train)0.9s + (eval)0.2s: train loss = 0.20049 = (mf)0.19862 + (embed)0.00187, recall = 0.14017, ndcg = 0.32160, precision = 0.28823, map = 0.07554
Epoch 11 (train)0.9s: train loss = 0.19690 = (mf)0.19487 + (embed)0.00203
Epoch 12 (train)0.9s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,98,4.818642
1,1,56,4.624581
2,1,475,4.576685
3,1,117,4.502943
4,1,144,4.497185
5,1,250,4.440380
6,1,919,4.420310
7,1,405,4.379748
8,1,423,4.376181
9,1,183,4.347711


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.111001
NDCG:	0.408241
Precision@K:	0.365748
Recall@K:	0.187918


## Number of layers = 2

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=2,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.1s: train loss = 0.47221 = (mf)0.47200 + (embed)0.00022
Epoch 2 (train)0.8s: train loss = 0.24568 = (mf)0.24500 + (embed)0.00067
Epoch 3 (train)0.8s: train loss = 0.22197 = (mf)0.22114 + (embed)0.00083
Epoch 4 (train)0.7s: train loss = 0.21058 = (mf)0.20963 + (embed)0.00095
Epoch 5 (train)0.8s + (eval)0.3s: train loss = 0.19347 = (mf)0.19241 + (embed)0.00106, recall = 0.14227, ndcg = 0.32367, precision = 0.29141, map = 0.07600
Epoch 6 (train)0.8s: train loss = 0.18353 = (mf)0.18235 + (embed)0.00118
Epoch 7 (train)0.8s: train loss = 0.17370 = (mf)0.17241 + (embed)0.00129
Epoch 8 (train)0.8s: train loss = 0.16640 = (mf)0.16501 + (embed)0.00140
Epoch 9 (train)0.8s: train loss = 0.16076 = (mf)0.15926 + (embed)0.00150
Epoch 10 (train)0.7s + (eval)0.2s: train loss = 0.15926 = (mf)0.15767 + (embed)0.00159, recall = 0.15204, ndcg = 0.34287, precision = 0.31188, map = 0.08280
Epoch 11 (train)0.8s: train loss = 0.15846 = (mf)0.15679 + (embed)0.00167
Epoch 12 (train)0.7s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,919,5.651679
1,1,403,5.551999
2,1,250,5.526236
3,1,91,5.359904
4,1,144,5.331599
5,1,275,5.330316
6,1,189,5.226426
7,1,431,5.206126
8,1,455,5.077051
9,1,588,5.063225


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.116749
NDCG:	0.415117
Precision@K:	0.369353
Recall@K:	0.192752


## Embedding size = 128

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          embed_size=128
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.4s: train loss = 0.42649 = (mf)0.42613 + (embed)0.00036
Epoch 2 (train)1.0s: train loss = 0.25816 = (mf)0.25740 + (embed)0.00077
Epoch 3 (train)0.9s: train loss = 0.23171 = (mf)0.23068 + (embed)0.00102
Epoch 4 (train)1.0s: train loss = 0.21643 = (mf)0.21522 + (embed)0.00122
Epoch 5 (train)0.9s + (eval)0.3s: train loss = 0.20699 = (mf)0.20561 + (embed)0.00138, recall = 0.13976, ndcg = 0.31516, precision = 0.28335, map = 0.07318
Epoch 6 (train)1.0s: train loss = 0.19018 = (mf)0.18860 + (embed)0.00158
Epoch 7 (train)1.0s: train loss = 0.18085 = (mf)0.17905 + (embed)0.00180
Epoch 8 (train)1.0s: train loss = 0.17025 = (mf)0.16825 + (embed)0.00200
Epoch 9 (train)1.0s: train loss = 0.16630 = (mf)0.16411 + (embed)0.00219
Epoch 10 (train)1.0s + (eval)0.2s: train loss = 0.15923 = (mf)0.15686 + (embed)0.00237, recall = 0.15136, ndcg = 0.34447, precision = 0.30774, map = 0.08602
Epoch 11 (train)1.0s: train loss = 0.15490 = (mf)0.15239 + (embed)0.00252
Epoch 12 (train)1.0s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,423,5.976227
1,1,91,5.649329
2,1,405,5.356187
3,1,919,5.339531
4,1,588,5.309151
5,1,433,5.275554
6,1,403,5.261531
7,1,432,5.150821
8,1,56,5.063169
9,1,568,5.045245


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.117577
NDCG:	0.419296
Precision@K:	0.374761
Recall@K:	0.193959


## Embedding size = 32

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          embed_size=32
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.2s: train loss = 0.51397 = (mf)0.51382 + (embed)0.00015
Epoch 2 (train)0.7s: train loss = 0.29497 = (mf)0.29441 + (embed)0.00055
Epoch 3 (train)0.7s: train loss = 0.26023 = (mf)0.25954 + (embed)0.00069
Epoch 4 (train)0.7s: train loss = 0.24162 = (mf)0.24080 + (embed)0.00082
Epoch 5 (train)0.7s + (eval)0.3s: train loss = 0.23057 = (mf)0.22963 + (embed)0.00094, recall = 0.12915, ndcg = 0.29708, precision = 0.26649, map = 0.06719
Epoch 6 (train)0.7s: train loss = 0.22343 = (mf)0.22239 + (embed)0.00104
Epoch 7 (train)0.7s: train loss = 0.21757 = (mf)0.21645 + (embed)0.00112
Epoch 8 (train)0.8s: train loss = 0.21970 = (mf)0.21851 + (embed)0.00118
Epoch 9 (train)0.7s: train loss = 0.21457 = (mf)0.21333 + (embed)0.00124
Epoch 10 (train)0.7s + (eval)0.2s: train loss = 0.20432 = (mf)0.20301 + (embed)0.00132, recall = 0.13493, ndcg = 0.30947, precision = 0.27731, map = 0.07094
Epoch 11 (train)0.7s: train loss = 0.20515 = (mf)0.20376 + (embed)0.00138
Epoch 12 (train)0.7s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,98,5.400963
1,1,56,5.146634
2,1,204,4.753932
3,1,238,4.718017
4,1,423,4.527374
5,1,144,4.496536
6,1,28,4.464203
7,1,405,4.418415
8,1,69,4.400910
9,1,655,4.337403


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.109146
NDCG:	0.400512
Precision@K:	0.358112
Recall@K:	0.184278


# **Ablation Study**

# Dropout = 0.1

L2 Regularization = 0.0

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.000 # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.5s: train loss = 0.46578 = (mf)0.46578 + (embed)0.00000
Epoch 2 (train)0.9s: train loss = 0.27583 = (mf)0.27583 + (embed)0.00000
Epoch 3 (train)1.0s: train loss = 0.24743 = (mf)0.24743 + (embed)0.00000
Epoch 4 (train)0.9s: train loss = 0.23029 = (mf)0.23029 + (embed)0.00000
Epoch 5 (train)0.9s + (eval)1.0s: train loss = 0.22071 = (mf)0.22071 + (embed)0.00000, recall = 0.13081, ndcg = 0.29661, precision = 0.26585, map = 0.06725
Epoch 6 (train)0.9s: train loss = 0.21596 = (mf)0.21596 + (embed)0.00000
Epoch 7 (train)0.9s: train loss = 0.20992 = (mf)0.20992 + (embed)0.00000
Epoch 8 (train)0.9s: train loss = 0.19953 = (mf)0.19953 + (embed)0.00000
Epoch 9 (train)0.9s: train loss = 0.18842 = (mf)0.18842 + (embed)0.00000
Epoch 10 (train)0.9s + (eval)0.2s: train loss = 0.17928 = (mf)0.17928 + (embed)0.00000, recall = 0.14730, ndcg = 0.33284, precision = 0.30329, map = 0.07831
Epoch 11 (train)0.9s: train loss = 0.17104 = (mf)0.17104 + (embed)0.00000
Epoch 12 (train)0.9s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,98,5.482670
1,1,919,5.222923
2,1,423,5.219283
3,1,69,5.187056
4,1,408,5.180061
5,1,403,5.174940
6,1,405,5.133166
7,1,91,5.075123
8,1,582,5.057949
9,1,28,5.049814


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.117221
NDCG:	0.420160
Precision@K:	0.373171
Recall@K:	0.193042


L2 Regularization = 0.0001

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.0001 # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.4s: train loss = 0.46632 = (mf)0.46607 + (embed)0.00024
Epoch 2 (train)0.9s: train loss = 0.27984 = (mf)0.27920 + (embed)0.00064
Epoch 3 (train)0.9s: train loss = 0.24628 = (mf)0.24547 + (embed)0.00081
Epoch 4 (train)0.9s: train loss = 0.23294 = (mf)0.23197 + (embed)0.00097
Epoch 5 (train)0.9s + (eval)0.3s: train loss = 0.22447 = (mf)0.22336 + (embed)0.00110, recall = 0.12909, ndcg = 0.29692, precision = 0.26575, map = 0.06710
Epoch 6 (train)0.9s: train loss = 0.21859 = (mf)0.21738 + (embed)0.00121
Epoch 7 (train)0.9s: train loss = 0.20841 = (mf)0.20709 + (embed)0.00132
Epoch 8 (train)0.9s: train loss = 0.19767 = (mf)0.19622 + (embed)0.00145
Epoch 9 (train)0.9s: train loss = 0.18702 = (mf)0.18542 + (embed)0.00160
Epoch 10 (train)0.9s + (eval)0.2s: train loss = 0.17480 = (mf)0.17304 + (embed)0.00176, recall = 0.14608, ndcg = 0.33305, precision = 0.30191, map = 0.07861
Epoch 11 (train)0.9s: train loss = 0.17110 = (mf)0.16918 + (embed)0.00192
Epoch 12 (train)0.9s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,432,5.463423
1,1,588,5.325284
2,1,91,5.176545
3,1,423,5.050926
4,1,98,4.976696
5,1,408,4.857317
6,1,455,4.788647
7,1,144,4.734303
8,1,204,4.620701
9,1,69,4.579630


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.116167
NDCG:	0.416462
Precision@K:	0.369777
Recall@K:	0.192483


L2 Regularization = 0.0005

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.0005 # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.4s: train loss = 0.46508 = (mf)0.46484 + (embed)0.00024
Epoch 2 (train)0.9s: train loss = 0.28421 = (mf)0.28357 + (embed)0.00063
Epoch 3 (train)0.9s: train loss = 0.24773 = (mf)0.24692 + (embed)0.00080
Epoch 4 (train)0.9s: train loss = 0.22829 = (mf)0.22730 + (embed)0.00099
Epoch 5 (train)0.9s + (eval)0.3s: train loss = 0.21923 = (mf)0.21810 + (embed)0.00113, recall = 0.13137, ndcg = 0.30282, precision = 0.26819, map = 0.06975
Epoch 6 (train)0.9s: train loss = 0.21374 = (mf)0.21251 + (embed)0.00123
Epoch 7 (train)0.9s: train loss = 0.20191 = (mf)0.20056 + (embed)0.00135
Epoch 8 (train)0.9s: train loss = 0.19828 = (mf)0.19681 + (embed)0.00148
Epoch 9 (train)0.9s: train loss = 0.18507 = (mf)0.18345 + (embed)0.00161
Epoch 10 (train)0.9s + (eval)0.2s: train loss = 0.17931 = (mf)0.17755 + (embed)0.00177, recall = 0.14713, ndcg = 0.33373, precision = 0.30329, map = 0.07902
Epoch 11 (train)0.9s: train loss = 0.17080 = (mf)0.16888 + (embed)0.00192
Epoch 12 (train)0.9s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,91,6.189005
1,1,423,5.415013
2,1,250,5.392582
3,1,144,5.386692
4,1,408,5.306597
5,1,588,5.282079
6,1,919,5.264353
7,1,405,5.239684
8,1,56,5.231057
9,1,204,5.223225


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.119439
NDCG:	0.421800
Precision@K:	0.375822
Recall@K:	0.196670


# Dropout = 0.25

L2 Regularization = 0.0

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.000, # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.7s: train loss = 0.43004 = (mf)0.43004 + (embed)0.00000
Epoch 2 (train)1.1s: train loss = 0.28745 = (mf)0.28745 + (embed)0.00000
Epoch 3 (train)1.1s: train loss = 0.24860 = (mf)0.24860 + (embed)0.00000
Epoch 4 (train)1.1s: train loss = 0.22786 = (mf)0.22786 + (embed)0.00000
Epoch 5 (train)1.1s + (eval)0.3s: train loss = 0.22238 = (mf)0.22238 + (embed)0.00000, recall = 0.12787, ndcg = 0.29717, precision = 0.26363, map = 0.06738
Epoch 6 (train)1.1s: train loss = 0.21764 = (mf)0.21764 + (embed)0.00000
Epoch 7 (train)1.1s: train loss = 0.20348 = (mf)0.20348 + (embed)0.00000
Epoch 8 (train)1.1s: train loss = 0.19269 = (mf)0.19269 + (embed)0.00000
Epoch 9 (train)1.1s: train loss = 0.18101 = (mf)0.18101 + (embed)0.00000
Epoch 10 (train)1.1s + (eval)0.3s: train loss = 0.17247 = (mf)0.17247 + (embed)0.00000, recall = 0.14727, ndcg = 0.33517, precision = 0.30615, map = 0.07927
Epoch 11 (train)1.1s: train loss = 0.16747 = (mf)0.16747 + (embed)0.00000
Epoch 12 (train)1.1s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,919,5.483072
1,1,91,5.458663
2,1,455,5.217731
3,1,423,5.201761
4,1,405,5.187254
5,1,250,5.146599
6,1,640,5.086217
7,1,83,5.043797
8,1,408,4.924489
9,1,652,4.869185


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.121333
NDCG:	0.424306
Precision@K:	0.377731
Recall@K:	0.198118


L2 Regularization = 0.0001

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.0001, # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.8s: train loss = 0.42557 = (mf)0.42519 + (embed)0.00038
Epoch 2 (train)1.2s: train loss = 0.29158 = (mf)0.29088 + (embed)0.00071
Epoch 3 (train)1.1s: train loss = 0.25394 = (mf)0.25296 + (embed)0.00098
Epoch 4 (train)1.1s: train loss = 0.23355 = (mf)0.23227 + (embed)0.00128
Epoch 5 (train)1.1s + (eval)0.3s: train loss = 0.22401 = (mf)0.22250 + (embed)0.00151, recall = 0.12609, ndcg = 0.29389, precision = 0.26161, map = 0.06590
Epoch 6 (train)1.1s: train loss = 0.21571 = (mf)0.21403 + (embed)0.00168
Epoch 7 (train)1.1s: train loss = 0.20666 = (mf)0.20480 + (embed)0.00186
Epoch 8 (train)1.1s: train loss = 0.19443 = (mf)0.19234 + (embed)0.00209
Epoch 9 (train)1.1s: train loss = 0.18345 = (mf)0.18108 + (embed)0.00237
Epoch 10 (train)1.1s + (eval)0.3s: train loss = 0.17140 = (mf)0.16875 + (embed)0.00265, recall = 0.15075, ndcg = 0.34092, precision = 0.30891, map = 0.08251
Epoch 11 (train)1.1s: train loss = 0.16635 = (mf)0.16344 + (embed)0.00291
Epoch 12 (train)1.1s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,275,5.149033
1,1,403,5.132240
2,1,91,5.084153
3,1,640,5.061930
4,1,423,5.034047
5,1,919,4.841353
6,1,405,4.785663
7,1,432,4.760918
8,1,582,4.738551
9,1,588,4.718047


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.117092
NDCG:	0.419531
Precision@K:	0.372004
Recall@K:	0.193749


L2 Regularization = 0.0005

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.0005, # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.8s: train loss = 0.43393 = (mf)0.43209 + (embed)0.00184
Epoch 2 (train)1.1s: train loss = 0.28546 = (mf)0.28186 + (embed)0.00360
Epoch 3 (train)1.1s: train loss = 0.25053 = (mf)0.24539 + (embed)0.00514
Epoch 4 (train)1.1s: train loss = 0.23295 = (mf)0.22653 + (embed)0.00642
Epoch 5 (train)1.1s + (eval)0.3s: train loss = 0.23160 = (mf)0.22440 + (embed)0.00720, recall = 0.12882, ndcg = 0.29677, precision = 0.26607, map = 0.06634
Epoch 6 (train)1.1s: train loss = 0.22137 = (mf)0.21346 + (embed)0.00792
Epoch 7 (train)1.1s: train loss = 0.21155 = (mf)0.20274 + (embed)0.00880
Epoch 8 (train)1.1s: train loss = 0.20273 = (mf)0.19292 + (embed)0.00981
Epoch 9 (train)1.1s: train loss = 0.19303 = (mf)0.18208 + (embed)0.01095
Epoch 10 (train)1.1s + (eval)0.3s: train loss = 0.18379 = (mf)0.17161 + (embed)0.01218, recall = 0.15021, ndcg = 0.33952, precision = 0.30976, map = 0.08066
Epoch 11 (train)1.1s: train loss = 0.18047 = (mf)0.16739 + (embed)0.01308
Epoch 12 (train)1.1s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,98,5.631125
1,1,588,5.412727
2,1,56,5.356334
3,1,423,5.150239
4,1,204,5.148867
5,1,91,5.130593
6,1,432,5.067955
7,1,144,5.057610
8,1,83,5.034612
9,1,28,4.800485


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.112445
NDCG:	0.410911
Precision@K:	0.367020
Recall@K:	0.186597


# Dropout = 0.50

L2 Regularization = 0.0

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.000, # l2 regularization for embedding parameters,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)2.0s: train loss = 0.42795 = (mf)0.42795 + (embed)0.00000
Epoch 2 (train)1.2s: train loss = 0.30287 = (mf)0.30287 + (embed)0.00000
Epoch 3 (train)1.2s: train loss = 0.27267 = (mf)0.27267 + (embed)0.00000
Epoch 4 (train)1.2s: train loss = 0.24442 = (mf)0.24442 + (embed)0.00000
Epoch 5 (train)1.2s + (eval)0.3s: train loss = 0.23184 = (mf)0.23184 + (embed)0.00000, recall = 0.12517, ndcg = 0.29344, precision = 0.26013, map = 0.06624
Epoch 6 (train)1.2s: train loss = 0.22600 = (mf)0.22600 + (embed)0.00000
Epoch 7 (train)1.2s: train loss = 0.22133 = (mf)0.22133 + (embed)0.00000
Epoch 8 (train)1.2s: train loss = 0.21365 = (mf)0.21365 + (embed)0.00000
Epoch 9 (train)1.2s: train loss = 0.20292 = (mf)0.20292 + (embed)0.00000
Epoch 10 (train)1.2s + (eval)0.2s: train loss = 0.19140 = (mf)0.19140 + (embed)0.00000, recall = 0.14221, ndcg = 0.32686, precision = 0.29682, map = 0.07644
Epoch 11 (train)1.2s: train loss = 0.17903 = (mf)0.17903 + (embed)0.00000
Epoch 12 (train)1.2s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,91,5.400868
1,1,98,5.346801
2,1,405,5.323782
3,1,56,5.222795
4,1,117,5.205643
5,1,403,5.197556
6,1,250,5.103917
7,1,475,5.084016
8,1,423,5.083714
9,1,455,5.059406


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.117167
NDCG:	0.418116
Precision@K:	0.372322
Recall@K:	0.194535


L2 Regularization = 0.0001

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.0001, # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)2.0s: train loss = 0.43150 = (mf)0.43112 + (embed)0.00038
Epoch 2 (train)1.2s: train loss = 0.31095 = (mf)0.31028 + (embed)0.00067
Epoch 3 (train)1.2s: train loss = 0.28010 = (mf)0.27921 + (embed)0.00090
Epoch 4 (train)1.2s: train loss = 0.24594 = (mf)0.24467 + (embed)0.00127
Epoch 5 (train)1.2s + (eval)0.3s: train loss = 0.23262 = (mf)0.23101 + (embed)0.00161, recall = 0.12265, ndcg = 0.28579, precision = 0.25323, map = 0.06322
Epoch 6 (train)1.2s: train loss = 0.22431 = (mf)0.22245 + (embed)0.00186
Epoch 7 (train)1.2s: train loss = 0.22128 = (mf)0.21923 + (embed)0.00205
Epoch 8 (train)1.2s: train loss = 0.21407 = (mf)0.21181 + (embed)0.00226
Epoch 9 (train)1.2s: train loss = 0.20413 = (mf)0.20164 + (embed)0.00249
Epoch 10 (train)1.2s + (eval)0.2s: train loss = 0.19392 = (mf)0.19113 + (embed)0.00279, recall = 0.14172, ndcg = 0.32782, precision = 0.29502, map = 0.07727
Epoch 11 (train)1.2s: train loss = 0.18434 = (mf)0.18122 + (embed)0.00311
Epoch 12 (train)1.2s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,423,5.316727
1,1,98,5.302406
2,1,56,5.110453
3,1,91,5.109349
4,1,403,4.997742
5,1,588,4.976910
6,1,204,4.926675
7,1,433,4.917244
8,1,405,4.879293
9,1,432,4.792632


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.115448
NDCG:	0.412973
Precision@K:	0.365217
Recall@K:	0.187879


L2 Regularization = 0.0005

In [ ]:
hparams = prepare_hparams('/content/recommenders/recommenders/models/deeprec/config/lightgcn.yaml',
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                          decay=0.000, # l2 regularization for embedding parameters
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)2.0s: train loss = 0.43160 = (mf)0.43160 + (embed)0.00000
Epoch 2 (train)1.2s: train loss = 0.30431 = (mf)0.30431 + (embed)0.00000
Epoch 3 (train)1.2s: train loss = 0.27066 = (mf)0.27066 + (embed)0.00000
Epoch 4 (train)1.2s: train loss = 0.23723 = (mf)0.23723 + (embed)0.00000
Epoch 5 (train)1.2s + (eval)0.4s: train loss = 0.22855 = (mf)0.22855 + (embed)0.00000, recall = 0.12608, ndcg = 0.29095, precision = 0.25843, map = 0.06665
Epoch 6 (train)1.2s: train loss = 0.22338 = (mf)0.22338 + (embed)0.00000
Epoch 7 (train)1.2s: train loss = 0.21806 = (mf)0.21806 + (embed)0.00000
Epoch 8 (train)1.2s: train loss = 0.21201 = (mf)0.21201 + (embed)0.00000
Epoch 9 (train)1.2s: train loss = 0.20073 = (mf)0.20073 + (embed)0.00000
Epoch 10 (train)1.2s + (eval)0.2s: train loss = 0.18964 = (mf)0.18964 + (embed)0.00000, recall = 0.13864, ndcg = 0.32395, precision = 0.29152, map = 0.07607
Epoch 11 (train)1.2s: train loss = 0.18271 = (mf)0.18271 + (embed)0.00000
Epoch 12 (train)1.2s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head(10)

,userID,itemID,prediction
0,1,98,5.535970
1,1,91,5.481869
2,1,423,5.136875
3,1,432,5.103333
4,1,204,5.082915
5,1,56,5.064419
6,1,405,5.006147
7,1,238,4.987628
8,1,919,4.954514
9,1,183,4.901851


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.119930
NDCG:	0.421403
Precision@K:	0.374125
Recall@K:	0.194712
